In [119]:
import torch
import math
x = torch.tensor([[1/3,1/3,1/3],[1.,.0,.0],[.1,.4,.5],[.1,.1,.8]], dtype=torch.float32)
batch_size = 4
label_size = 3
threshold = 0.75

In [120]:

entr = torch.ones(batch_size, requires_grad=False) #[batch_size] 
pred = torch.zeros((batch_size,label_size), requires_grad=False) #[batch_size, label_size]
total_remain_idx = torch.ones(batch_size, dtype=torch.bool) #[batch_size]

In [121]:
x_out = x
#for idx in range(self.num_layer):
for idx in range(5):
    # f forward
    #x_out = self.layer_forward(x_out, idx, mask)
    # return form b/h
    #y_out = self.bridge_return(x_out, idx, mask)
    y_out = torch.rand(x_out.size())

    y_entr = torch.sum(torch.special.entr(y_out),dim=-1) / math.log(y_out.size(-1))

    total_remain_idx = entr>threshold
    entr[total_remain_idx] = y_entr
    pred[total_remain_idx,:] = y_out
    
    remain_idx = y_entr>threshold
    print(f"\n{idx}")
    print("total",total_remain_idx)
    print("remain",remain_idx)
    print("entr",entr)
    print("pred",pred)
    print("x",x_out)
    x_out = x_out[remain_idx,:]
    
    
    print("x_remain",x_out)
    if x_out.size(0) == 0:
        break



0
total tensor([True, True, True, True])
remain tensor([False,  True, False, False])
entr tensor([0.6301, 0.8748, 0.6448, 0.7467])
pred tensor([[0.1885, 0.6032, 0.9243],
        [0.6999, 0.4833, 0.4469],
        [0.3601, 0.6562, 0.9337],
        [0.1973, 0.4342, 0.8504]])
x tensor([[0.3333, 0.3333, 0.3333],
        [1.0000, 0.0000, 0.0000],
        [0.1000, 0.4000, 0.5000],
        [0.1000, 0.1000, 0.8000]])
x_remain tensor([[1., 0., 0.]])

1
total tensor([False,  True, False, False])
remain tensor([False])
entr tensor([0.6301, 0.6062, 0.6448, 0.7467])
pred tensor([[0.1885, 0.6032, 0.9243],
        [0.4177, 0.0882, 0.9086],
        [0.3601, 0.6562, 0.9337],
        [0.1973, 0.4342, 0.8504]])
x tensor([[1., 0., 0.]])
x_remain tensor([], size=(0, 3))


In [53]:
y_out[y_entr>threshold]

tensor([[0.3333, 0.3333, 0.3333],
        [0.1000, 0.4000, 0.5000]])

In [88]:
total_remain_idx = torch.ones(batch_size, dtype=torch.bool)


In [96]:
total_remain_idx[[True,False,False,False]].size(0) == 0

1

In [9]:
from datasets import load_dataset
train_data = load_dataset('dbpedia_14', split='train')
valid_data = load_dataset('dbpedia_14', split='test[:50%]')
test_data = load_dataset('dbpedia_14', split='test[50%:]')

Using custom data configuration default
Reusing dataset ag_news (/home/user/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
Using custom data configuration default
Reusing dataset ag_news (/home/user/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
Using custom data configuration default
Reusing dataset ag_news (/home/user/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


In [25]:
from datasets import load_dataset
train_data = load_dataset('dbpedia_14', split='train')
test_data = load_dataset('dbpedia_14', split='test').shuffle(seed=35)

split_size = int(test_data.num_rows*0.5)
valid_data = test_data[:split_size]
test_data = test_data[split_size:]


Reusing dataset dbpedia_14 (/home/user/.cache/huggingface/datasets/dbpedia_14/dbpedia_14/2.0.0/01dab9e10d969eadcdbc918be5a09c9190a24caeae33b10eee8f367a1e3f1f0c)
Reusing dataset dbpedia_14 (/home/user/.cache/huggingface/datasets/dbpedia_14/dbpedia_14/2.0.0/01dab9e10d969eadcdbc918be5a09c9190a24caeae33b10eee8f367a1e3f1f0c)
Loading cached shuffled indices for dataset at /home/user/.cache/huggingface/datasets/dbpedia_14/dbpedia_14/2.0.0/01dab9e10d969eadcdbc918be5a09c9190a24caeae33b10eee8f367a1e3f1f0c/cache-68e93457c310e291.arrow


In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split

class_num = 2
df = pd.read_csv('./IMDB_Dataset.csv')
df['cleaned_reviews'] = df['review']
corpus = [word for text in df['cleaned_reviews'] for word in text.split()]
text = [t for t in df['cleaned_reviews']]
label = []
for t in df['sentiment']:
    if t == 'negative':
        label.append(1)
    else:
        label.append(0)

clean_train, clean_test, train_label, test_label = train_test_split(text, label, test_size=0.2, random_state=35)
clean_valid, clean_test, valid_label, test_label = train_test_split(clean_test, test_label, test_size=0.5, random_state=35)

In [40]:
import pandas
print(pandas.DataFrame(train_label).value_counts())
print(pandas.DataFrame(valid_label).value_counts())
print(pandas.DataFrame(test_label).value_counts())
print(len(clean_train))
print(len(clean_valid))
print(len(clean_test))
print(len(train_label))
print(len(valid_label))
print(len(test_label))

0    20050
1    19950
dtype: int64
1    4035
0    3965
dtype: int64
1    1015
0     985
dtype: int64
40000
8000
2000
40000
8000
2000


In [28]:
import pandas
pandas.DataFrame(test_data['label']).value_counts()


2     2577
3     2550
13    2549
5     2538
7     2532
6     2531
12    2513
10    2482
0     2472
1     2456
4     2453
9     2453
8     2449
11    2445
dtype: int64

In [19]:
import pandas
pandas.DataFrame(test_data['label']).value_counts()


3    1541
0    1532
1    1507
2    1500
dtype: int64

In [22]:
import pandas
pandas.DataFrame(test_data['label']).value_counts()

2    400
1    393
0    368
3    359
dtype: int64